# Scripts Mosaicos

In [1]:
import ee
import geemap
import os
import folium

In [2]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Mosaicos

### Patagonia

In [3]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
work_dir = os.path.expanduser('../../Data/shapes/')
norte = os.path.join(work_dir, 'area_norte.shp')
patagonia_norte = geemap.shp_to_ee(norte)
Map.addLayer(patagonia_norte, {}, 'norte')

In [5]:
work_dir = os.path.expanduser('../../Data/shapes/')
centro = os.path.join(work_dir, 'area_centro.shp')
patagonia_centro = geemap.shp_to_ee(centro)
Map.addLayer(patagonia_centro, {}, 'centro')

In [6]:
work_dir = os.path.expanduser('../../Data/shapes/')
sur = os.path.join(work_dir, 'area_sur.shp')
patagonia_sur = geemap.shp_to_ee(sur)
Map.addLayer(patagonia_grande, {}, 'sur')

## Funciones

In [7]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [8]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [9]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [12]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [13]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [14]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [15]:
def Agua1986(image):
    return image.updateMask(ndwi1986.gte(0.1))

In [16]:
def Agua1985(image):
    return image.updateMask(ndwi1985.gte(0.1))

In [17]:
def Agua1984(image):
    return image.updateMask(ndwi1984.gte(0.1))

In [18]:
def Ndvi1986(image):
    return image.updateMask(ndvi1986.gt(0.05))

In [19]:
def Ndvi1985(image):
    return image.updateMask(ndvi1985.lt(0.2))

In [20]:
def Ndvi1984(image):
    return image.updateMask(ndvi1984.lt(0.2))

In [21]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

### Funciones SUR

In [22]:
def Ndvi1986_sur(image):
    return image.updateMask(ndvi1986.gt(0))

In [23]:
def Agua1986_sur(image):
    return image.updateMask(ndwi1986.lt(0.4))

## Colecciones

### Verano 1986

In [24]:
start_date = '1986-10-01'
end_date = '1987-03-31'

l5_1986 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [26]:
coleccion1986 = l5_1986.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

#### Verano Sur 1984-1987

In [27]:
start_date = '1984-10-01'
end_date = '1987-03-31'

l5_1986_sur = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [28]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [29]:
coleccionsur = l5_1986_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 1985

In [30]:
start_date = '1985-10-01'
end_date = '1986-03-31'

l5_1985 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [31]:
coleccion1985 = l5_1985.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 1984

In [32]:
start_date = '1984-10-01'
end_date = '1985-03-31'

l5_1984 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [33]:
coleccion1984 = l5_1984.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

## Quality Mosaic NDVI

In [34]:
NDVI_1986 = coleccion1986.qualityMosaic('NDVI')
NDVI_1985 = coleccion1985.qualityMosaic('NDVI')
NDVI_1984 = coleccion1984.qualityMosaic('NDVI')

In [35]:
NDVI_sur = coleccionsur.qualityMosaic('NDVI')

In [36]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(NDVI_1986, visParams, 'Greenest 1986')
Map.addLayer(NDVI_1985, visParams, 'Greenest 1985')
Map.addLayer(NDVI_1984, visParams, 'Greenest 1984')
Map.addLayer(NDVI_sur, visParams, 'Greenest sur')
Map.centerObject(patagonia_grande, 6)
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position'], widget=HBox…

In [37]:
ndvi1986 = NDVI_1986.select('NDVI')
ndvi1985 = NDVI_1985.select('NDVI')
ndvi1984 = NDVI_1984.select('NDVI')
ndvisur = NDVI_sur.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [38]:
Map.addLayer(ndvi1986, {'palette': palette}, 'NDVI_1986')
Map.addLayer(ndvi1985, {'palette': palette}, 'NDVI_1985')
Map.addLayer(ndvi1984, {'palette': palette}, 'NDVI_1984')
Map.addLayer(ndvisur,{'palette': palette}, 'NDVI_sur')
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position'], widget=HBox…

## Agua

### Verano 1986

In [39]:
aquast_1986 = coleccion1986.qualityMosaic('NDWI')
aquast_sur = coleccionsur.qualityMosaic('NDWI')

### Verano 1985

In [40]:
aquast_1985 = coleccion1985.qualityMosaic('NDWI')

### Verano 1984

In [41]:
aquast_1984 = coleccion1984.qualityMosaic('NDWI')

### Quality Mosaic NDWI

In [42]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(aquast_1986, visParams, 'Agua 1986')
Map2.addLayer(aquast_1985, visParams, 'Agua 1985')
Map2.addLayer(aquast_1984, visParams, 'Agua 1984')
Map2.addLayer(aquast_sur, visParams, 'Agua sur')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position'], widget=HBox…

In [43]:
ndwi1986 = aquast_1986.select('NDWI')
ndwi_viz = ['00FFFF', '0000FF']
ndwi1985 = aquast_1985.select('NDWI')
ndwi1984 = aquast_1984.select('NDWI')
ndwisur = aquast_sur.select('NDWI')
Map2.addLayer(ndwi1986, {'palette': ndwi_viz}, 'NDWI_1986')
Map2.addLayer(ndwi1985, {'palette': ndwi_viz}, 'NDWI_1985')
Map2.addLayer(ndwi1984, {'palette': ndwi_viz}, 'NDWI_1984')
Map2.addLayer(ndwisur, {'palette': ndwi_viz}, 'NDWI_sur')
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position'], widget=HBox…

In [44]:
ndwi_rgb = ndwi1986.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

In [45]:
ndwi_mask = ndwi1986.updateMask(ndwi1986.gte(0.4))

In [46]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, {} , 'NDWI sin tierra')
display(map_ndwi_mask)

In [47]:
ndwi_mask1986 = ndwi1986.updateMask(ndwi1986.gte(0.4))

In [48]:
map_ndwi_mask1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask1986.add_ee_layer(ndwi_mask1986, {}, 'NDWI sin tierra')
display(map_ndwi_mask1986)

## Máscaras

### Verano 2021

#### Nubes 50

In [49]:
coleccion86 = coleccion1986.map(Cloud50)
greenest86 = coleccion86.qualityMosaic('NDVI')
map_1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986.add_ee_layer(greenest86, vizParams, 'Mapa sin Nubes')
display(map_1986)

#### Nubes 30

In [50]:
coleccion86 = coleccion1986.map(Cloud30)
greenest86 = coleccion86.qualityMosaic('NDVI')
map_1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986.add_ee_layer(greenest86, vizParams, 'Mapa sin Nubes')
display(map_1986)

#### Nubes Sur

In [51]:
coleccionsur_ = coleccionsur.map(Cloud30)
greenestsur_ = coleccionsur_.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(greenestsur_, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [52]:
coleccionsur2 = coleccionsur.map(Cloud50)
greenestsur2 = coleccionsur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenestsur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

#### Filtro de agua

In [53]:
coleccion86_ = coleccion1986.map(Cloud50)
coleccion86_ = coleccion86_.map(Agua1986)
greenest86_ = coleccion86_.qualityMosaic('NDVI')
map_1986_ = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986_.add_ee_layer(greenest86_, vizParams, 'Mapa sin Nubes')
display(map_1986_)

In [54]:
agua_1986 = coleccion1986.map(Cloud50)
agua_1986 = agua_1986.map(Agua1986)
aquast86_ = agua_1986.qualityMosaic('NDWI')
map_agua1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua1986.add_ee_layer(aquast86_, vizParams, 'Mapa sin Nubes')
display(map_agua1986)

In [55]:
coleccion86 = coleccion1986.map(Cloud50)
greenest86 = coleccion86.qualityMosaic('NDVI')
map_1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986.add_ee_layer(greenest86, vizParams, 'Mapa sin Nubes')
display(map_1986)

#### Filtro NDVI

In [56]:
coleccion86 = coleccion1986.map(Cloud50)
coleccion86 = coleccion86.map(Ndvi1986)
greenest86_2 = coleccion86.qualityMosaic('NDVI')
map_1986_2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986_2.add_ee_layer(greenest86_2, vizParams, 'Mapa sin Nubes')
display(map_1986_2)

In [57]:
coleccion_sur2 = coleccionsur.map(Cloud30)
coleccion_sur2 = coleccion_sur2.map(Ndvi1986_sur)
coleccion_sur2 = coleccion_sur2.map(Agua1986_sur)
greenest_sur2 = coleccion_sur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenest_sur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

### Verano 2020

In [58]:
coleccion85 = coleccion1985.map(Cloud50)
# coleccion2020 = coleccion2020.map(Agua)
greenest85 = coleccion85.qualityMosaic('NDVI')
map_1985 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1985.add_ee_layer(greenest85, vizParams, 'Mapa sin nubes')
display(map_1985)

In [59]:
agua_1985 = coleccion1985.map(Agua1985)
aquast85_ = agua_1985.qualityMosaic('NDWI')
map_agua1985 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua1985.add_ee_layer(aquast85_, vizParams, 'Mapa sin Nubes')
display(map_agua1985)

### Verano 2019

In [60]:
coleccion84 = coleccion1984.map(Cloud50)
# coleccion2019 = coleccion2019.map(Agua)
greenest84 = coleccion84.qualityMosaic('NDVI')
map_1984 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1984.add_ee_layer(greenest84, vizParams, 'Mapa sin nubes')
display(map_1984)

## Mosaicos

In [61]:
Map3.addLayer(ndvi1986, {'palette': palette}, 'NDVI_1986')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [62]:
mosaic = ee.ImageCollection([greenestsur_,greenest86]).mosaic()

In [63]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [64]:
mosaic2 = ee.ImageCollection([mosaic, aquast85_]).mosaic()

In [65]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [66]:
mosaic3 = ee.ImageCollection([mosaic2, greenest86_2]).mosaic()

In [67]:
Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [68]:
mosaic4 = ee.ImageCollection([mosaic3, greenest_sur2]).mosaic()

In [69]:
Map3.addLayer(mosaic4, vizParams, 'Mosaico 4')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [70]:
mosaic5 = ee.ImageCollection([mosaic4,aquast85_]).mosaic()

In [71]:
Map3.addLayer(mosaic5, vizParams, 'Mosaico 5')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…